# Codigo EEMRIO by Martínez-Hernando


This information is necessary in Python, for activate the funtions 
If they are not installed is necessary to do it, write pip install XXXXX. If pip doesn't work put pip3, or pip3.11 (search on internet)

Activate generical information

In [1]:
import pandas as pd #To real CSV and dataframes
import numpy as np #To work with matrix
from scipy import io, integrate, linalg, signal #Google told me, I don´t know what is it
from scipy.sparse.linalg import cg, eigs #Google told me, I don´t know what is it

To create list

In [2]:
from typing import List, Tuple, Dict, Callable, Iterable, Union
import itertools

## Importing EXIOBASE 3.3.18 data
We will only import:
- $Z$, the intermediate consumption,
- $Y$, the final demand,
- $F$ and $F_Y$, the environmental extensions, only for air emissions.

In [3]:
exiobase_folder="P:/Universidad/Exiobase/" #Put here your folder where you have download Exiobase with /, no with \

Z (Intermediate products)

In [4]:
Z_raw=pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_by_product_technology.csv", #You should have this name of file, if not change it
                    index_col = [0,1,2,3,4],
                    header = [0,1,2,3])
Z_raw.index.names=["region","sector","code1","code2","units"]
Z_raw.head() #To see the first lines

AU  \
                                                   Cultivation of paddy rice   
                                                                       i01.a   
                                                                      A_PARI   
region sector                  code1 code2  units                              
AU     Paddy rice              p01.a C_PARI tonnes                       0.0   
       Wheat                   p01.b C_WHEA tonnes                       0.0   
       Cereal grains nec       p01.c C_OCER tonnes                       0.0   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                       0.0   
       Oil seeds               p01.e C_OILS tonnes                       0.0   

                                                                         \
                                                   Cultivation of wheat   
                                                                  i01.b   
                                                                 A_WHEA   
region sector                  code1 code2  units                         
AU     Paddy rice              p01.a C_PARI tonnes                  0.0   
       Wheat                   p01.b C_WHEA tonnes                  0.0   
       Cereal grains nec       p01.c C_OCER tonnes                  0.0   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                  0.0   
       Oil seeds               p01.e C_OILS tonnes                  0.0   

                                                                                     \
                                                   Cultivation of cereal grains nec   
                                                                              i01.c   
                                                                             A_OCER   
region sector                  code1 code2  units                                     
AU     Paddy rice              p01.a C_PARI tonnes                          0.00000   
       Wheat                   p01.b C_WHEA tonnes                          0.00000   
       Cereal grains nec       p01.c C_OCER tonnes                      70942.96547   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                          0.00000   
       Oil seeds               p01.e C_OILS tonnes                          0.00000   

                                                                                           \
                                                   Cultivation of vegetables, fruit, nuts   
                                                                                    i01.d   
                                                                                   A_FVEG   
region sector                  code1 code2  units                                           
AU     Paddy rice              p01.a C_PARI tonnes                                0.00000   
       Wheat                   p01.b C_WHEA tonnes                                0.00000   
       Cereal grains nec       p01.c C_OCER tonnes                                0.00000   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                            18266.75262   
       Oil seeds               p01.e C_OILS tonnes                                0.00000   

                                                                             \
                                                   Cultivation of oil seeds   
                                                                      i01.e   
                                                                     A_OILS   
region sector                  code1 code2  units                             
AU     Paddy rice              p01.a C_PARI tonnes                      0.0   
       Wheat                   p01.b C_WHEA tonnes                      0.0   
       Cereal grains nec       p01.c C_OCER tonnes                      0.0   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                      0.0   
       Oi

In [5]:
Z_reindexed=Z_raw.droplevel([2,3,4], axis=0).copy() #To remove useless rows
Z_reindexed.index.names=["region","sector"]
Z_reindexed.columns=Z_reindexed.index
Z_reindexed.head()


region                                 AU                          \
sector                         Paddy rice Wheat Cereal grains nec   
region sector                                                       
AU     Paddy rice                     0.0   0.0           0.00000   
       Wheat                          0.0   0.0           0.00000   
       Cereal grains nec              0.0   0.0       70942.96547   
       Vegetables, fruit, nuts        0.0   0.0           0.00000   
       Oil seeds                      0.0   0.0           0.00000   

region                                                            \
sector                         Vegetables, fruit, nuts Oil seeds   
region sector                                                      
AU     Paddy rice                              0.00000       0.0   
       Wheat                                   0.00000       0.0   
       Cereal grains nec                       0.00000       0.0   
       Vegetables, fruit, nuts             18266.75262       0.0   
       Oil seeds                               0.00000       0.0   

region                                                                    \
sector                         Sugar cane, sugar beet Plant-based fibers   
region sector                                                              
AU     Paddy rice                                 0.0               0.00   
       Wheat                                      0.0               0.00   
       Cereal grains nec                          0.0               0.00   
       Vegetables, fruit, nuts                    0.0               0.00   
       Oil seeds                                  0.0        -1206065.87   

region                                                                 ...  \
sector                         Crops nec        Cattle           Pigs  ...   
region sector                                                          ...   
AU     Paddy rice                    0.0  0.000000e+00       0.000000  ...   
       Wheat                         0.0  6.791574e+05  156083.893300  ...   
       Cereal grains nec             0.0  2.277000e+06  823314.442100  ...   
       Vegetables, fruit, nuts       0.0  1.552051e+03     255.322381  ...   
       Oil seeds                     0.0  0.000000e+00       0.000000  ...   

region                                                    WM  \
sector                         Paper for treatment: landfill   
region sector                                                  
AU     Paddy rice                                        0.0   
       Wheat                                             0.0   
       Cereal grains nec                                 0.0   
       Vegetables, fruit, nuts                           0.0   
       Oil seeds                                         0.0   

region                                                                \
sector                         Plastic waste for treatment: landfill   
region sector                                                          
AU     Paddy rice                                                0.0   
       Wheat                                                     0.0   
       Cereal grains nec                                         0.0   
       Vegetables, fruit, nuts                                   0.0   
       Oil seeds                                                 0.0   

region                                                                              \
sector                         Inert/metal/hazardous waste for treatment: landfill   
region sector                                                                        
AU     Paddy rice                                                             0.0    
       Wheat                                                                  0.0    
       Cereal grains nec                                                      0.0    
       Vegetables, fruit, nuts                                

In [6]:
exio3318_Z=Z_reindexed

Y (Final demand)

In [7]:
Y_raw = pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_FD.csv", #You should have this name of file, if not change it
                    index_col = [0,1,2,3,4],
                    header = [0,1,2,3])

Y_reindexed=Y_raw.droplevel([2,3], axis=1).copy() #To remove useless columns
Y_reindexed.index=Z_reindexed.index
Y_reindexed.columns.names=["region", "sector"] #To name the columns
exio3318_Y=Y_reindexed

F and F_Y (Extensions, total emissions per activity)

In [8]:
F_raw = pd.read_excel(exiobase_folder + "MR_HIOT_2011_v3_3_18_extensions.xlsx", #This data is emissions of intermediate production
                      sheet_name="Emiss_act", #You should have this name of file, if not change it.
                      index_col=[0,1,2],
                      header=[0,1,2,3])
F_raw.columns=Z_reindexed.columns

F_Y_raw=pd.read_excel(exiobase_folder + "MR_HIOT_2011_v3_3_18_extensions.xlsx", #This data is emissions of final demand
                      sheet_name="Emiss_FD", #You should have this name of file, if not change it.
                      index_col=[0,1,2],
                      header=[0,1,2,3])
F_Y_raw.columns=Y_reindexed.columns

F_raw.head()
F_Y_raw.head()
exio3318_emissions_F=F_raw
exio3318_emissions_F_Y=F_Y_raw

In [9]:
F_Y_raw.head()

region                                                                     AU  \
sector                            Final consumption expenditure by households   
Carbon dioxide, fossil tonnes air                                4.760370e+07   
N2O                    tonnes air                                9.248053e+02   
CH4                    tonnes air                                4.907784e+04   
HFCs                   tonnes air                                0.000000e+00   
PFCs                   tonnes air                                0.000000e+00   

region                                                                                                                  \
sector                            Final consumption expenditure by non-profit organisations serving households (NPISH)   
Carbon dioxide, fossil tonnes air                                           0.000000                                     
N2O                    tonnes air                                           0.000000                                     
CH4                    tonnes air                                         200.749203                                     
HFCs                   tonnes air                                           0.000000                                     
PFCs                   tonnes air                                           0.000000                                     

region                                                                         \
sector                            Final consumption expenditure by government   
Carbon dioxide, fossil tonnes air                                    0.000000   
N2O                    tonnes air                                    0.000000   
CH4                    tonnes air                                  343.422972   
HFCs                   tonnes air                                    0.000000   
PFCs                   tonnes air                                    0.000000   

region                                                           \
sector                            Gross fixed capital formation   
Carbon dioxide, fossil tonnes air                             0   
N2O                    tonnes air                             0   
CH4                    tonnes air                             0   
HFCs                   tonnes air                             0   
PFCs                   tonnes air                             0   

region                                                                         \
sector                            Changes in inventories Changes in valuables   
Carbon dioxide, fossil tonnes air                      0                    0   
N2O                    tonnes air                      0                    0   
CH4                    tonnes air                      0                    0   
HFCs                   tonnes air                      0                    0   
PFCs                   tonnes air                      0                    0   

region                                                                     AT  \
sector                            Final consumption expenditure by households   
Carbon dioxide, fossil tonnes air                                2.287570e+07   
N2O                    tonnes air                                6.054108e+02   
CH4                    tonnes air                                1.750280e+04   
HFCs                   tonnes air                                0.000000e+00   
PFCs                   tonnes air                                0.000000e+00   

region                                                                                                                  \
sector                            Final consumption expenditure by non-profit organisations serving households (NPISH)   
Carbon dioxide, fossil tonnes air                                                  0                                     
N2O                    tonnes air    

In [10]:
X_raw = pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_principal_production.csv", #You should have this name of file, if not change it
                    header = [0,1,2,3,4,5,6,7])

In [11]:
X_reindexed=X_raw.droplevel([2,3,4,5,6,7], axis=1).copy()
X_reindexed.columns=exio3318_Z.columns #To rename the columns
X_reindexed.index=['Total production'] #To rename the index
exio3318_X=X_reindexed

In [12]:
A=exio3318_Z/exio3318_X.squeeze() #To calculate the matrix A
A=A.fillna(0) #To fill the NaN values with 0
n= len(A.index) #To know the number of rows
Ident=np.eye(n) #To create the identity matrix
I_A=Ident-A #To calculate the matrix I-A

In [13]:
print(I_A.shape) #To know the shape of the matrix
I_A.isna().any().any()
print(type(I_A)) #To know the type of the matrix

(7872, 7872)
<class 'pandas.core.frame.DataFrame'>


In [24]:
exio3318_emissions_S=exio3318_emissions_F/exio3318_X.squeeze()
exio3318_emissions_S=exio3318_emissions_S.fillna(0)
exio3318_emissions_S=exio3318_emissions_S*1000

b_data=np.array([exio3318_emissions_S.iloc[0,:],exio3318_emissions_S.iloc[1,:],exio3318_emissions_S.iloc[2,:]])

# Importation for BW2.5

Libraries for brightway2.5

In [15]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw_processing as bwp
from bw_processing.io_helpers import generic_directory_filesystem
import fsspec
from pathlib import Path
import json
import bw2io as bi

print('bw2data version: ',bd.__version__)
print('bw2io version: ',bi.__version__)
print('bw2calc version: ',bc.__version__)

bd.projects
bd.projects.set_current('exiobase_dpk')

if any("forwast" in db for db in bd.databases): #to confirm biosphere is present
    print('forwast is already present in the project.')
else:
    bi.remote.get_projects()
    bi.remote.install_project('forwast','exiobase_dpk', overwrite_existing=True)

bw2data version:  (4, 4, 4)
bw2io version:  0.9.9
bw2calc version:  2.0.2
forwast is already present in the project.


To confirm biosphere is well installed

In [16]:
methods_installed=len(bd.methods) #This number is not 1, if it is 1; try to run in an other kernel
print(bd.databases)
if methods_installed==1:
    print('You have a problem in brightway installation, only one mtehod have been added. It will be deleted and intalled again')

Databases dictionary with 2 object(s):
	ecoinvent-3.9.1-biosphere
	forwast


# Creation of matrix needed

Index needed

In [17]:
countries=exio3318_Z.columns.get_level_values(0)
products=exio3318_Z.columns.get_level_values(1)

nick_names=[f"{coun}_{prod}" for coun, prod in zip(countries, products)]
nick_names=np.array(nick_names)
units=np.array(Z_raw.index.get_level_values("units")) #To see the units
print(nick_names)
print(len(nick_names))
print(units)
print(type(units))

['AU_Paddy rice' 'AU_Wheat' 'AU_Cereal grains nec' ...
 'WM_Other services (93)'
 'WM_Private households with employed persons (95)'
 'WM_Extra-territorial organizations and bodies']
7872
['tonnes' 'tonnes' 'tonnes' ... 'Meuro' 'Meuro' 'Meuro']
<class 'numpy.ndarray'>


In [18]:
n=exio3318_Z.columns.shape[0]
nick=list(range(0,n))
nick[-1]
#nick=np.array([nick],dtype=bwp.INDICES_DTYPE)

7871

In [19]:
CO2=20001 #It is necessary to put a numeric value of CO2 emissions. Activities goes to 0 to 7871, so here the number is higher
N2O=20002
CH4=20003
emissions=[CO2, N2O, CH4]

## Technosphere matrix (L value)

Values

In [20]:
tec_data_matrix=I_A.values
tec_data=tec_data_matrix.flatten()
tec_data.shape

(61968384,)

Index

In [21]:
tec_index=list(itertools.product(nick, nick))
tec_index=np.array(tec_index, dtype=bwp.INDICES_DTYPE)

In [32]:
tec_index.shape

(61968384,)

## Biosphere matrix

Values

In [33]:
b_data=np.array([exio3318_emissions_S.iloc[0,:],exio3318_emissions_S.iloc[1,:],exio3318_emissions_S.iloc[2,:]])
b_data=b_data.flatten()
b_data=b_data.squeeze()
b_data=np.nan_to_num(b_data)
print(b_data.shape)

(23616,)


Index

In [34]:
b_index=list(itertools.product(emissions, nick))
b_index=np.array([b_index],dtype=bwp.INDICES_DTYPE).T
b_index=b_index.squeeze()
print(b_index.shape)
b_index


(23616,)


array([(20001,    0), (20001,    1), (20001,    2), ..., (20003, 7869),
       (20003, 7870), (20003, 7871)],
      dtype=[('row', '<i8'), ('col', '<i8')])

## Method

In [35]:
c_data=np.array([1, 273, 29.8]) #For EF V3.1
print(c_data)
c_index=np.array([(CO2, CO2),(N2O,N2O),(CH4,CH4)] ,dtype=bwp.INDICES_DTYPE)
print(c_index.shape)


[  1.  273.   29.8]
(3,)


# Creation of datapackage

In [36]:
fs = generic_directory_filesystem(dirpath=Path(exiobase_folder))

dp_static = bwp.create_datapackage(
    fs=fs,
    name="exiobase_static",
    matrix_serialize_format_type=bwp.MatrixSerializeFormat.NUMPY
)

In [37]:
dp_static.add_persistent_vector(
    matrix='technosphere_matrix',
    indices_array=tec_index,
    data_array=tec_data,
)
dp_static.add_persistent_vector(
    matrix='biosphere_matrix',
    indices_array=b_index,
    data_array=b_data,
)
dp_static.add_persistent_vector(
    matrix='characterization_matrix',
    indices_array=c_index,
    data_array=c_data,
)

In [38]:
dp_static.finalize_serialization()

# LCA

In [39]:
lca = bc.LCA(
    demand={0: 1},  
    data_objs=[dp_static],
)
lca.lci()
lca.lcia()
lca.score
print(lca.score)

1934.6759717028049
